In [1]:
import pandas as pd
import torch
import joblib
from preprocessing import *
from JREP import JREP
from PC_transformer import PCTransformer
from RNN import RNN
from LSTM import LSTM
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, root_mean_squared_error

In [2]:
df = pd.read_csv(r'output_csv\ANL-Intrepid-2009-1.swf.csv')

In [3]:
feature_columns = ['requested_processors', 'requested_time', 'submit_time', 'wait_time', 'user_id', 'queue_id']
target_column = 'run_time'

# Hyperparameters
input_dim = len(feature_columns)    # Example input dimension
d_model = 64      # Model dimension
num_heads = 8     # Number of attention heads
d_ff = 256        # Feed-forward dimension
num_layers = 3    # Number of transformer blocks
output_dim = 1    # Output dimension
batch_size = 128  # Batch size
seq_len = 20      # Length of data group
num_epochs = 10   # Number of training epochs
num_hidden = 64   # Number of hidden units
dropout = 0.2     # Dropout rate
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


dataloaders, scaler_seq = prepare_data_seq(df, feature_columns, target_column, statuss=-1, seq_len=seq_len, batch_size=batch_size)
X_train, X_test, Y_train, Y_test, scaler = prepare_data(df, feature_columns, target_column, statuss=-1)

In [4]:
PCTransformer_model = PCTransformer(input_dim, d_model, num_heads, d_ff, num_layers, output_dim, dropout).to(device)
PCTransformer_model.load_state_dict(torch.load('models/best_pc_transformer_model.pth', map_location=device))
PCTransformer_rmse, PCTransformer_mae, PCTransformer_mse, PCTransformer_r2 = PCTransformer_model.evaluate_model(PCTransformer_model, dataloaders['test'], device, scaler_seq)

rnn_model = RNN(input_dim, num_hidden, num_layers, dropout).to(device)
rnn_model.load_state_dict(torch.load('models/best_rnn_model.pth', map_location=device))
rnn_rmse, rnn_mae, rnn_mse, rnn_r2 = rnn_model.evaluate_model(rnn_model, dataloaders['test'], scaler_seq, input_dim)

lstm_model = LSTM(input_dim, num_hidden, num_layers, dropout).to(device)
lstm_model.load_state_dict(torch.load('models/best_lstm_model.pth', map_location=device))
lstm_rmse, lstm_mae, lstm_mse, lstm_r2 = lstm_model.evaluate_model(lstm_model, dataloaders['test'], scaler_seq, input_dim)

JREP_model = JREP()
stacking_model = joblib.load('models/stacking_model.pkl')
JREP_model.stacking_model = stacking_model
Y_pred = JREP_model.predict(X_test, scaler, len(feature_columns))
JREP_rmse, JREP_mae, JREP_mse, JREP_r2 = JREP_model.evaluate_model(Y_test, Y_pred, scaler, len(feature_columns))

In [5]:
print(f"PCTransformer RMSE: {PCTransformer_rmse:.4f}")
print(f"PCTransformer MAE: {PCTransformer_mae:.4f}")
print(f"PCTransformer MSE: {PCTransformer_mse:.4f}")
print(f"PCTransformer R2: {PCTransformer_r2:.4f}")

PCTransformer RMSE: 7117.8422
PCTransformer MAE: 4020.2324
PCTransformer MSE: 50663678.1636
PCTransformer R2: 0.5935


In [6]:
print(f"JREP RMSE: {JREP_rmse:.4f}")
print(f"JREP MAE: {JREP_mae:.4f}")
print(f"JREP MSE: {JREP_mse:.4f}")
print(f"JREP R2: {JREP_r2:.4f}")

JREP RMSE: 5298.4297
JREP MAE: 1209.1404
JREP MSE: 28073356.8164
JREP R2: 0.7140


In [7]:
print(f"RNN RMSE: {rnn_rmse:.4f}")
print(f"RNN MAE: {rnn_mae:.4f}")
print(f"RNN MSE: {rnn_mse:.4f}")
print(f"RNN R2: {rnn_r2:.4f}")

RNN RMSE: 7753.3992
RNN MAE: 4641.0103
RNN MSE: 60115198.8636
RNN R2: 0.5175


In [8]:
print(f'LSTM RMSE: {lstm_rmse:.4f}')
print(f'LSTM MAE: {lstm_mae:.4f}')
print(f'LSTM MSE: {lstm_mse:.4f}')
print(f'LSTM R2: {lstm_r2:.4f}')

LSTM RMSE: 7088.6480
LSTM MAE: 3902.8084
LSTM MSE: 50248930.1810
LSTM R2: 0.5967
